In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
hotel_details=pd.read_csv("C:/Users/princ/Desktop/WEBD/WEBD/Hotel_details.csv/Hotel_details.csv",delimiter=',')
hotel_rooms=pd.read_csv("C:/Users/princ/Desktop/WEBD/WEBD/Hotel_details.csv/Hotel_Room_attributes.csv",delimiter=',')
hotel_cost=pd.read_csv("C:/Users/princ/Desktop/WEBD/WEBD/Hotel_details.csv/Hotel_Room_attributes.csv",delimiter=',')
hotel_rooms.columns
hotel_details.head()
hotel_rooms.head()
hotel_cost.head()
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')
hotel.columns
del hotel['hotelid']
del hotel['url']
del hotel['curr']
del hotel['Source']
hotel.columns

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude'],
      dtype='object')

In [5]:
hotel_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99153 entries, 0 to 108047
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotelid       99153 non-null  int64  
 1   hotelname     99153 non-null  object 
 2   address       99153 non-null  object 
 3   city          99153 non-null  object 
 4   country       99153 non-null  object 
 5   propertytype  99153 non-null  object 
 6   starrating    99153 non-null  int64  
 7   latitude      99153 non-null  float64
 8   longitude     99153 non-null  float64
 9   Source        99153 non-null  int64  
 10  url           99153 non-null  object 
 11  curr          99153 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 9.8+ MB


In [6]:
hotel_details.describe()

,hotelid,starrating,latitude,longitude,Source
count,9.915300e+04,99153.000000,99153.000000,99153.000000,99153.000000
mean,9.391405e+05,3.174982,46.598677,8.558842,2.972164
std,1.698268e+06,0.667139,5.302805,9.042434,1.404069
min,3.100000e+01,2.000000,0.000000,-111.908966,2.000000
25%,9.283800e+04,3.000000,43.280579,2.161577,2.000000
50%,2.942020e+05,3.000000,46.567860,9.201611,2.000000
75%,8.094740e+05,4.000000,50.522709,13.759882,5.000000
max,9.061192e+06,4.000000,69.536102,34.408731,5.000000


In [7]:
hotel_rooms.describe()

,hotelcode
count,1.610540e+05
mean,1.171866e+06
std,2.144665e+06
min,3.100000e+01
25%,8.279800e+04
50%,1.560250e+05
75%,8.092970e+05
max,8.977298e+06


In [8]:
hotel_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165873 entries, 0 to 165872
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               165873 non-null  int64 
 1   hotelcode        165873 non-null  int64 
 2   roomamenities    161054 non-null  object
 3   roomtype         165873 non-null  object
 4   ratedescription  161054 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.3+ MB


In [9]:
def calc(hotel, room_no):
    guests_no = []
    for roomtype in hotel['roomtype']:
        temp = roomtype.lower().split()
        flag = 0
        for word in temp:
            for room, guests in room_no:
                if word == room:
                    guests_no.append(guests)
                    flag = 1
                    break
            if flag == 1:
                break
        if flag == 0:
            guests_no.append(2)
    return guests_no
room_no = [
    ('king', 2),
    ('queen', 2),
    ('triple', 3),
    ('master', 3),
    ('family', 4),
    ('murphy', 2),
    ('quad', 4),
    ('double-double', 4),
    ('mini', 2),
    ('studio', 1),
    ('junior', 2),
    ('apartment', 4),
    ('double', 2),
    ('twin', 2),
    ('double-twin', 4),
    ('single', 1),
    ('disabled', 1),
    ('accessible', 1),
    ('suite', 2),
    ('one', 2)
]
guests_no = calc(hotel, room_no)
hotel['guests_no'] = guests_no
hotel['roomamenities'] = hotel['roomamenities'].str.replace(': ;', ',')

In [10]:
def requirementbased(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['city','hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)
requirementbased('London',3,'I need air conditioned room. I should have an alarm clock.')

,city,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
284,london,Comfort Inn St Pancras - Kings Cross,Triple,3,3,2-5 St. Chad's Street,"air conditioning,alarm clock,coffee/tea maker,...","Room size: 20 m²/215 ft², Non-smoking, Shower,...",4
221,london,Gatwick White House Hotel,Standard Triple Room,3,3,"52 Church Road, Horley, Surrey,Near Gatwick Ai...","air conditioning,alarm clock,carpeting,closet,...","Room size: 18 m²/194 ft², Nature view, Non-smo...",4
203,london,Little Foxes Hotel,Triple Room En-Suite,3,3,"Charlwood Road, Crawley, West sussex","additional toilet,air conditioning,alarm clock...","Room size: 30 m²/323 ft², Country/region view,...",4
205,london,Best Western Chiswick Palace,Standard Triple Room,3,3,71-73 Chiswick High RoadUK,"air conditioning,alarm clock,carpeting,closet,...","Room size: 23 m²/248 ft², 1 single bed and 1 d...",4
429,london,Barrington Lodge,Deluxe Triple Room,3,4,Hunter Road,"air conditioning,alarm clock,carpeting,closet,...","Room size: 30 m²/323 ft², Shower, 3 single beds",4
426,london,Best Western Swiss Cottage,Triple Room,3,3,4 Adamson Road,"air conditioning,alarm clock,carpeting,closet,...","Room size: 25 m²/269 ft², Non-smoking, Shower,...",4
200,london,Langshott Manor - A Small Luxury Hotel,Master Suite,3,4,"Ladbroke Road, Gatwick","air conditioning,alarm clock,bathrobes,blackou...","Room size: 32 m²/344 ft², Garden view, Balcony...",4
198,london,The Park City Grand Plaza Kensington Hotel,Executive Triple Room,3,4,18-30 Lexham Gardens,"air conditioning,alarm clock,carpeting,closet,...","Room size: 26 m²/280 ft², City view, Non-smoki...",4
141,london,Brit Hotels London Court,Classic Triple Room,3,3,194-196 Earls Court Road,"air conditioning,alarm clock,carpeting,closet,...","Room size: 15 m²/161 ft², Shower, 1 single bed...",4
144,london,Princess Hotel,Standard Triple Room,3,2,35-39 Argyle Street,"air conditioning,alarm clock,carpeting,closet,...","Room size: 21 m²/226 ft², Non-smoking, Shower,...",4
